In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# IMPORT OF ALL REQUIRED LIBRARIES
%matplotlib inline

import time
import pylab as pl
from IPython import display
import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
import re
import urllib
import requests
from PIL import Image
from matplotlib import style
import matplotlib.gridspec as gridspec

#  SET CHART STYLES
style.use('bmh')

# CREATE A LIST OF WORDS TO NOT SHOW IN THE WORD CLOUD
stopwords = set(STOPWORDS)
stopwords.update(["https", "co", "x92", "ugpHT6k30c", "MexWPLPGfn", "x91Game", "x92m", "bartlebee24_", "ve"])

# IMPORT AN IMAGE FOR THE WORD CLOUD TO TAKE THE SHAPE OF
mask = np.array(Image.open(requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/DemDonkey.svg/1920px-DemDonkey.svg.png', stream=True).raw))

# DATAFILE PATHS FOR USE IN IMPORTS 
file_path = 'tweets_chart.csv'
file_path2 = "tweets.csv"

# CREATING THE FIGURE WITH MULTIPLE SUBPLOTS
fig = pl.figure()
fig.tight_layout()
fig.set_size_inches(50, 25) 

gs = gridspec.GridSpec(3,3)

ax1=fig.add_subplot(gs[:,0])
ax2=fig.add_subplot(gs[0,1:])
ax3=fig.add_subplot(gs[1,1:], sharex=ax2)
ax4=fig.add_subplot(gs[2,1:], sharex=ax2)
fig.show()

# RUN THE RECURSIVE LOOP UNTIL THE USER INTERRUPTS
while True:
    try:
# RECURSIVE IMPORT OF FRESH DATA BACKFILLING EMPTY DATAPOINTS DUE TO STREAM ERRORS        
        df = pd.read_csv(file_path) # DATA READ FOR CHARTS     
        dfx = df[["TWT_POST_DT", "POSSENTIMENT", "NEGSENTIMENT"]].fillna(method='ffill')                          # DATA FOR AVG SENT CHART
        dfy = df[["TWT_POST_DT", "POSCNT", "NEGCNT"]].replace(to_replace=0, method='ffill')                       # DATA FOR COUNT POS AND NEG TWEETS CHART
        dfv = df[["TWT_POST_DT", "FOLLOWER_CNT", "FRIENDS_CNT"]].replace(to_replace=0, method='ffill')            # DATA FOR THE FOLLOWERS FRIENDS CHART
        
        df2 = pd.read_csv(file_path2, encoding = "1252")   
        dfz = df2.replace('http://\S+|https://\S+', '', regex=True)                                               # DATA FOR WORD CLOUD CHART
        dfz.TWEET.replace("'", "")
        dfz = dfz.TWEET.values        
                
# PLOTTING WORD CLOUD OF HIGHEST FREQUENCY WORDS  
#        wordcloud = WordCloud(width=512, height=512, background_color = 'black', min_font_size=10, stopwords = stopwords, mask = mask, contour_width=3, contour_color='firebrick').generate(str(dfz).replace("'", ""))        
        wordcloud = WordCloud(width=512, height=512, background_color = 'black', min_font_size=10, stopwords = stopwords, contour_width=3, contour_color='firebrick').generate(str(dfz).replace("'", ""))        

        ax1.imshow(wordcloud, interpolation="bilinear", aspect="auto")
        ax1.set_title("Word Cloud by Frequency", fontsize=25)
        ax1.axis("off")     
        ax1.margins(x=0, y=0)
        
# PLOTTING AVERAGE POSITIVE AND NEGATIVE SENTIMENT 
        ax2.clear() # to clear the whole axes            
        ax2.set_title('Average Tweet Sentiment Over Time', fontsize=25)
        ax2.set_ylabel('Average Senitment Score', fontsize=25)
        ax2.set_xlabel('DateTime Segment', fontsize=25)  
        dfx.plot(x="TWT_POST_DT", kind="line", linewidth=3,  markersize=14, markeredgecolor='k', markeredgewidth=2.0, color=['g', 'r'], marker='o', fontsize=20, ax=ax2) 
        ax2.legend().set_visible(False)
        
# PLOTTING COUNT OF POSITIVE AND NEGATIVE TWEETS         
        ax3.clear() # to clear the whole axes  
        ax3.set_title("Total Positive and Negative Tweets Over Time", fontsize=25)
        ax3.set_ylabel('Total Tweets', fontsize=25)
        ax3.set_xlabel('DateTime Segment', fontsize=25)  
        dfy.plot(x="TWT_POST_DT", kind="bar", stacked=True, color=['g', 'r'], edgecolor='k', linewidth=3.0, fontsize=20, ax=ax3, sharex = ax2) 
        ax3.legend().set_visible(False)   
                
# PLOTTING THE TOTAL NUMBER OF FOLLOWERS AND FRIENDS DIRECT VISIBILITY OF USER'S TWEET      
        ax4.clear() # to clear the whole axes  
        ax4.set_title("Total Followers and Friends with Access to Tweet (millions)", fontsize=25)
        ax4.set_ylabel('Total Followers/Friends Reached', fontsize=25)
        ax4.set_xlabel('DateTime Segment', fontsize=25)  
        dfv.plot(x="TWT_POST_DT", kind="bar", stacked=True, color=['b', 'k'], edgecolor='k', linewidth=3.0, fontsize=20, ax=ax4, sharex = ax2) 
        ax4.legend().set_visible(False)   
             
# RESET THE VISUALIZATIONS FOR THE NEXT ITERATION
        display.display(pl.gcf())
        display.clear_output(wait=True)       
        time.sleep(3)
        pl.cla
    
# IF THE USER INTERRUPTS THE PROGRAM, END WITHOUT ERRORS        
    except KeyboardInterrupt:
        display.display(pl.clf())
        break